## Helmholtz Transmission Problem

Consider the TP for given parameters $\mu_i,\kappa_i$, $i=0,1$.

In [28]:
import bempp.api as bem
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt

Set the parameters. Here we consider the unit sphere.

In [ ]:
kExt = 0.4
kInt = 1
rhoExt = 1
rhoInt = 1

R = 1
h = 0.2
alpha = rhoInt / rhoExt

# Gmres
maxiter = 2000
restart = 200
tolerance = 1E-4


def rescale(A, alpha):
    """Rescale the 2x2 block operator matrix A"""
    A[0, 1] = A[0, 1] * alpha
    A[1, 0] = A[1, 0] * (1/alpha)
    return A





def dirichlet_fun(x, n, domain_index, result):
    result[0] = np.exp(1j * kExt * x[0])

def neumann_fun(x, n, domain_index, result):
    result[0] = 1j * kExt * np.exp(1j * kExt * x[0]) * n[0]



grid = bem.shapes.sphere(h=h, r=R)
space = bem.function_space(grid, "P", 1)
N = int(space.global_dof_count)
print N, 'NP1'

AExt = bem.operators.boundary.helmholtz.multitrace_operator(grid, kExt)
I  = bem.operators.boundary.sparse.multitrace_identity(grid, spaces='linear')
AInt = bem.operators.boundary.helmholtz.multitrace_operator(grid, kInt)
STF = rescale(AInt, alpha) + AExt
dirichlet_grid_fun = bem.GridFunction(space, fun=dirichlet_fun)
neumann_grid_fun = bem.GridFunction(space, fun=neumann_fun)

rhs = [dirichlet_grid_fun, neumann_grid_fun]
rhs_sk =  rhs

print 'Solver First-Kind'

x, info, residuals = bem.linalg.gmres( STF * STF, STF * rhs, tol=tolerance, return_residuals=True, use_strong_form=True, maxiter=maxiter, restart=restart)


Recover the unknowns as GridFunctions:

In [ ]:
dtrace = bem.GridFunction(space,coefficients = x[:N])
ntrace = bem.GridFunction(space,coefficients = x[N:])

Plot the GMRES residual:

In [ ]:
plt.plot(np.log10(residuals), '--', linewidth=1.5, color= 'red')
plt.xlabel('Iteration count')
plt.ylabel('Relative l2-error (log)')
plt.legend()